In [19]:
import pandas as pd
import numpy as np
import scipy.sparse


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

In [13]:
mypath = "/home/mmarzec12/data/"
savepath = "/home/mmarzec12/models/mf/model_tuning/"

explicit = pd.read_csv(mypath+"explicit_train.csv")
validation = pd.read_csv(mypath+"leave_one_out_validation.csv")

In [18]:
# list with (user,item) tuples from validation set
validation_list = [(u,i) for u,i in zip(validation.user_name, validation.game_id)]
# dict with user:game key-value pairs from validation set
validation_dict = {u:i for u,i in zip(validation.user_name, validation.game_id)}

# unique games and users
unique_users = explicit.user_name.unique()
unique_games = explicit.game_id.unique()

# dictonaries to map users to unique ids and vice vers
us_to_ids = {u:i for i,u in enumerate(unique_users)}
ids_to_us = {i:u for i,u in enumerate(unique_users)}

# dictonaries to map games to unique ids and vice vers
gs_to_ids = {g:i for i,g in enumerate(unique_games)}
ids_to_gs = {i:g for i,g in enumerate(unique_games)}


# creating sparse matrix with data
row = [us_to_ids[us] for us in explicit.user_name]
col = [gs_to_ids[g] for g in explicit.game_id]
data = explicit.score

A = scipy.sparse.coo_matrix((data, (row, col)), shape=(len(unique_users), len(unique_games)))
#A = A.tocsr()

In [28]:
A.todok()[1,2]

6.0

In [38]:
class RecDataset(torch.utils.data.Dataset):
    
    def __init__(self, sparse_matrix):
        super().__init__()
        self.mat = sparse_matrix.tocoo()
        
    def __len__():
        return self.mat.nnz
        
    def __getitem__(self, index):
        row = self.mat.row[index]
        col = self.mat.col[index]
        val = self.mat.data[index]
        return (row, col), val

In [46]:
class BaseMF(nn.Module):
    
    def __init__(self, n_factors, n_users, n_items):
        super().__init__()
        self.user_biases = nn.Embedding(n_users, 1)
        self.item_biases = nn.Embedding(n_items, 1)
        self.user_embeddings = nn.Embedding(n_users, n_factors)
        self.item_embeddings = nn.Embedding(n_items, n_factors)
        
        
    def forward(self, user_id, item_id):
        return (self.user_embeddings(user_id) * self.item_embeddings(item_id)).sum()


In [51]:
model = BaseMF(n_factors=20, n_users=len(unique_users), n_items=len(unique_games))

loss = nn.MSELoss()
opt = torch.optim.SGD(model.parameters(), lr=1e-6)
train = RecDataset(A)

n_epochs = 5

for epoch in range(n_epochs):
    for x in train:
        ids, val = x
        val = torch.FloatTensor([val])
        row = torch.LongTensor([ids[0]])
        col = torch.LongTensor([ids[1]])
        
        
        opt.zero_grad()
        
        pred = model(row, col)
        l = loss(pred, val)
        print(l)
        loss.backward()
        
        opt.step()

tensor(52.8506, grad_fn=<MseLossBackward0>)


/home/mmarzec12/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


AttributeError: 'MSELoss' object has no attribute 'backward'

In [11]:
model.parameters

<bound method Module.parameters of MF(
  (user_biases): Embedding(1, 1)
  (item_biases): Embedding(1, 1)
  (user_embeddings): Embedding(1, 1)
  (item_embeddings): Embedding(1, 1)
)>

In [11]:
mf.requires_grad_ = False

False

In [ ]:
    
    
class ModelPipeline(nn.Module):
    
    def __init__(self):
        pass
        
        
    def fit(self):
        return
        
    def fit_epoch(self):
        return